In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [2]:
def get_bits(x, n):
    return [int(x) for x in '{:0{size}b}'.format(x, size=n)]

In [3]:
def generate_carry_part(constant, n):
    constant_bits = get_bits(constant, n)
    
    reg_A = QuantumRegister(n, name='a') # |a> - input register
    reg_G = QuantumRegister(n, name='g') # |g> - dirty ancilla register, state have to be restored
    reg_C = QuantumRegister(1, name='c') # |c> - output register - information about carry
    
    qc = QuantumCircuit(reg_A, reg_G, reg_C)

    qc.cx(reg_G[n - 1], reg_C)

    for i in range(n - 1, 0, -1):
        if constant_bits[n-(i+1)] == 1:
            qc.cx(reg_A[i], reg_G[i])
            qc.x(reg_A[i])
        qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])
        # qc.barrier(reg_A[:], reg_G[:], reg_C[:])

    if constant_bits[n-1] == 1:
        qc.cx(reg_A[0], reg_G[0])

    for i in range(1, n):
        qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])

    #qc.barrier(reg_A[:], reg_G[:], reg_C[:])

    qc.cx(reg_G[n - 1], reg_C)

    #qc.barrier(reg_A[:], reg_G[:], reg_C[:])

    for i in range(n - 1, 0, -1):
        qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])

    #qc.barrier(reg_A[:], reg_G[:], reg_C[:])

    if constant_bits[n-1] == 1:
        qc.cx(reg_A[0], reg_G[0])

    #qc.barrier(reg_A[:], reg_G[:], reg_C[:])

    for i in range(1, n):
        qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])
        if constant_bits[n-(i+1)] == 1:
            qc.x(reg_A[i])
            qc.cx(reg_A[i], reg_G[i])
        #qc.barrier(reg_A[:], reg_G[:], reg_C[:])
    
    return qc

In [4]:
# qc = generate_carry_part(11, 4)
# qc.draw(output='mpl')

In [5]:
def generate_init_part(a, g, n):
    a_bits = get_bits(a, n)
    g_bits = get_bits(g, n)
    
    qrA = QuantumRegister(n, name='a')
    qrG = QuantumRegister(n, name='g')
    qc = QuantumCircuit(qrA, qrG)
    
    for i in range(n):
        if a_bits[n-(i+1)] == 1:
            qc.x(qrA[i])

    for i in range(n):
        if g_bits[n-(i+1)] == 1:
            qc.x(qrG[i])

    qc.barrier(qrA[:], qrG[:])
    
    return qc

In [6]:
# qc_init = generate_init_part(6, 10, 4)
# qc_init.draw(output='mpl')

In [7]:
def generate_measure_part(n):
    qrA = QuantumRegister(n, name='a')
    qrG = QuantumRegister(n, name='g')
    qrC = QuantumRegister(1, name='c')
    crA = ClassicalRegister(n, name='aValue')
    crG = ClassicalRegister(n, name='gValue')
    crC = ClassicalRegister(1, name='cValue')

    qc = QuantumCircuit(qrA, qrG, qrC, crA, crG, crC)
    qc.barrier(qrA[:], qrG[:], qrC[:])
    qc.measure(qrA[:], crA[:])
    qc.measure(qrG[:], crG[:])
    qc.measure(qrC[:], crC[:])
    
    return qc

In [8]:
# qc_meas = generate_measure_part(4)
# qc_meas.draw(output='mpl')

In [9]:
def test_carry(a, constant, n, carry):
    expected_carry = int((a + constant) / (2 ** n))
    print(expected_carry, carry, expected_carry == carry)

In [10]:
test_carry(6, 11, 4, 1)

1 1 True


In [11]:
from qiskit import Aer
from qiskit import execute

backend = Aer.get_backend('qasm_simulator')

In [12]:
# """
# a = 6
# g = 10
# constant = 11
# n = 4
# """

In [13]:
def test(a, g, constant, n):
    init = generate_init_part(a, g, n)
    carry_gate = generate_carry_part(constant, n)
    meas = generate_measure_part(n)

    qc = init + carry_gate + meas

    job = execute(qc, backend, shots=1)
    result = job.result()
    values = list(result.get_counts(qc).keys())

    assert len(values) == 1
    values = [int(v, 2) for v in values[0].split(' ')]

    c_value = values[0]
    g_value = values[1]
    a_value = values[2]

    print(a, a_value, a == a_value)
    print(g, g_value, g == g_value)
    test_carry(a, constant, n, c_value)

In [14]:
test(6, 10, 11, 4)

<ipython-input-13-6aa9ef8a3bd6>:6: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = init + carry_gate + meas
/home/bartek/S10/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


6 6 True
10 10 True
1 1 True


In [15]:
# for n in range(4):
#     n = n + 1
#     N = 2 ** n
#     for a in range(N):
#         for g in range(N):
#             for c in range(N):
#                 print('---', a, g, c, '---')
#                 test(a, g, c, n)